In [1]:
import pandas as pd # 데이터
import numpy as np # 기본적인 기능
import matplotlib as mpl
import matplotlib.pyplot as plt # 시각화
import matplotlib.font_manager as fm # 한글폰트 적용
import seaborn as sns # 시각화

font_path = 'NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path,size=18)

In [2]:
from sklearn.model_selection import train_test_split # 훈련방식
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [3]:
import warnings
warnings.filterwarnings('ignore')
#오류 뜨는거 보기싫어서 삭제

In [4]:
card_df = pd.read_csv('train.csv') # 데이터 저장
card_df.head(2) # 데이터 상단 2개보기

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0


In [5]:
cardfull_df = card_df.fillna(value='unemployed') # occyp_type의 Nan 값을  비고용자로 변경
cardfull_df.head(2)

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,unemployed,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0


In [6]:
cardhangul = cardfull_df.rename(columns={'gender' : '성별',   # 이해하기 쉽게 칼럼을 한글로 변경
                      'car' : '차',
                      'reality' : '집',
                      'child_num' : '자녀 수',
                      'income_total' : '연간소득',
                      'income_type' : '소득분류',
                      'edu_type':'교육수준',
                      'family_type':'결혼여부',
                      'house_type':'생활방식',
                      'DAYS_BIRTH':'출생일',
                      'DAYS_EMPLOYED':'취직일',
                      'FLAG_MOBIL':'폰',
                      'work_phone':'업무용 폰',
                      'phone':'전화',
                      'email':'이메일',
                      'occyp_type' : '직업' ,
                      'family_size' : '가족규모',
                      'begin_month' : '카드발급일',
                      'credit':'신용등급'                        
                           })

In [7]:
def income (x):  # 소득분위 만들기위한 함수
    income = ''
    if x <= 3000 : income= "1분위"
    elif x <= 4100 : income= "2분위"
    elif x <= 5270 : income= "3분위"
    elif x <= 5700 : income= "4분위"
    elif x <= 7600 : income= "5분위"
    elif x <= 8550 : income= "6분위"
    elif x <= 11700 : income= "7분위"
    elif x <= 17550 : income= "8분위"
    else : income = "9분위"
    return income

In [8]:
cards = cardhangul.loc[:,'연간소득'] #연간소득 데이터만 따로 불러오기
income_card=cards*0.1
income_list=[]
for i  in income_card:
    income_list.append(income(i))

cardhangul['소득분위']=income_list
cardhangul.head(5)

,index,성별,차,집,자녀 수,연간소득,소득분류,교육수준,결혼여부,생활방식,...,취직일,폰,업무용 폰,전화,이메일,직업,가족규모,카드발급일,신용등급,소득분위
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,...,-4709,1,0,0,0,unemployed,2.0,-6.0,1.0,9분위
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,...,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0,9분위
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,...,-4434,1,0,1,0,Managers,2.0,-22.0,2.0,9분위
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,...,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0,9분위
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,...,-2105,1,0,0,0,Managers,2.0,-26.0,2.0,8분위


In [9]:
carddrop = cardhangul.drop(['카드발급일','출생일','index','취직일'],axis=1)
carddrop


#신용등급을 인트값으로 변경
#레이블로 숫자로 변경
# 필요없는값은 드랍 (출생일 ,카드발급일, 취직일)



,성별,차,집,자녀 수,연간소득,소득분류,교육수준,결혼여부,생활방식,폰,업무용 폰,전화,이메일,직업,가족규모,신용등급,소득분위
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,1,0,0,0,unemployed,2.0,1.0,9분위
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1,0,0,1,Laborers,3.0,1.0,9분위
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,1,0,1,0,Managers,2.0,2.0,9분위
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,1,0,1,0,Sales staff,2.0,0.0,9분위
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,1,0,0,0,Managers,2.0,2.0,8분위
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,1,0,0,0,Core staff,4.0,1.0,9분위
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,1,0,0,0,unemployed,2.0,2.0,9분위
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,1,0,0,0,Core staff,2.0,2.0,9분위
26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,1,0,0,0,Laborers,1.0,2.0,8분위


In [10]:
# 레이블 인코딩 : 데이터를 숫자값으로 변경하는것

In [11]:
pd.set_option('display.max_rows', 100 ) # 정수화 
encoder = LabelEncoder()
carddrop['성별']=encoder.fit_transform(carddrop['성별'].values)
carddrop['차']=encoder.fit_transform(carddrop['차'].values)
carddrop['집']=encoder.fit_transform(carddrop['집'].values)
carddrop['결혼여부']=encoder.fit_transform(carddrop['결혼여부'].values)
carddrop['교육수준']=encoder.fit_transform(carddrop['교육수준'].values)
carddrop['생활방식']=encoder.fit_transform(carddrop['생활방식'].values)
carddrop['직업']=encoder.fit_transform(carddrop['직업'].values)
carddrop['소득분위']=encoder.fit_transform(carddrop['소득분위'].values)
carddrop['소득분류']=encoder.fit_transform(carddrop['소득분류'].values)
carddrop=carddrop.astype({'연간소득':'int','가족규모':'int','신용등급':'int'})
carddrop.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   성별      26457 non-null  int32
 1   차       26457 non-null  int32
 2   집       26457 non-null  int32
 3   자녀 수    26457 non-null  int64
 4   연간소득    26457 non-null  int32
 5   소득분류    26457 non-null  int32
 6   교육수준    26457 non-null  int32
 7   결혼여부    26457 non-null  int32
 8   생활방식    26457 non-null  int32
 9   폰       26457 non-null  int64
 10  업무용 폰   26457 non-null  int64
 11  전화      26457 non-null  int64
 12  이메일     26457 non-null  int64
 13  직업      26457 non-null  int32
 14  가족규모    26457 non-null  int32
 15  신용등급    26457 non-null  int32
 16  소득분위    26457 non-null  int32
dtypes: int32(12), int64(5)
memory usage: 2.2 MB


In [343]:
y_card_df = carddrop['신용등급']
x_card_df = carddrop.drop(columns=['소득분위','신용등급'])

X_train,X_test,y_train,y_test=train_test_split(x_card_df,y_card_df,test_size=0.12,random_state=700)

In [344]:

#3개의 알고리즘을 위한 사이킷런의 클래스 생성
dt_clf = DecisionTreeClassifier(random_state=700)
rf_clf = RandomForestClassifier(random_state=700)
lr_clf = LogisticRegression()

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))


DecisionTreeClassifier 정확도: 0.6510
RandomForestClassifier 정확도: 0.6847
LogisticRegression 정확도: 0.6498


In [280]:
testdf=pd.read_csv('test.csv')

In [283]:
testdf2 = testdf.rename(columns={'gender' : '성별',   # 이해하기 쉽게 칼럼을 한글로 변경
                      'car' : '차',
                      'reality' : '집',
                      'child_num' : '자녀 수',
                      'income_total' : '연간소득',
                      'income_type' : '소득분류',
                      'edu_type':'교육수준',
                      'family_type':'결혼여부',
                      'house_type':'생활방식',
                      'DAYS_BIRTH':'출생일',
                      'DAYS_EMPLOYED':'취직일',
                      'FLAG_MOBIL':'폰',
                      'work_phone':'업무용 폰',
                      'phone':'전화',
                      'email':'이메일',
                      'occyp_type' : '직업' ,
                      'family_size' : '가족규모',
                      'begin_month' : '카드발급일'  
                           })

In [290]:
testdf3 = testdf2.loc[:,'연간소득'] #연간소득 데이터만 따로 불러오기
income_card=testdf3*0.1
income_list=[]
for i  in income_card:
    income_list.append(income(i))

testdf2['소득분위']=income_list
testdf2.head(5)

,index,성별,차,집,자녀 수,연간소득,소득분류,교육수준,결혼여부,생활방식,출생일,취직일,폰,업무용 폰,전화,이메일,직업,가족규모,카드발급일,소득분위
0,26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0,7분위
1,26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0,8분위
2,26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0,5분위
3,26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0,7분위
4,26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0,9분위


In [291]:
testdf4 = testdf2.drop(['카드발급일','출생일','index','취직일'],axis=1)
testdf4

,성별,차,집,자녀 수,연간소득,소득분류,교육수준,결혼여부,생활방식,폰,업무용 폰,전화,이메일,직업,가족규모,소득분위
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,1,0,1,0,NaN,2.0,7분위
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,1,0,1,0,Core staff,2.0,8분위
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,1,1,1,0,Laborers,2.0,5분위
3,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,1,1,0,0,Drivers,2.0,7분위
4,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,1,1,0,0,Managers,2.0,9분위
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,Y,Y,0,202500.0,Working,Incomplete higher,Married,House / apartment,1,1,1,0,Accountants,2.0,9분위
9996,M,Y,Y,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,1,1,0,0,Laborers,2.0,9분위
9997,F,N,Y,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,1,0,0,0,Medicine staff,2.0,9분위
9998,F,Y,N,0,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,1,0,1,0,NaN,2.0,9분위


In [293]:
pd.set_option('display.max_rows', 100 ) # 정수화 
encoder = LabelEncoder()
testdf4['성별']=encoder.fit_transform(testdf4['성별'].values)
testdf4['차']=encoder.fit_transform(testdf4['차'].values)
testdf4['집']=encoder.fit_transform(testdf4['집'].values)
testdf4['결혼여부']=encoder.fit_transform(testdf4['결혼여부'].values)
testdf4['교육수준']=encoder.fit_transform(testdf4['교육수준'].values)
testdf4['생활방식']=encoder.fit_transform(testdf4['생활방식'].values)
testdf4['직업']=encoder.fit_transform(testdf4['직업'].values)
testdf4['소득분위']=encoder.fit_transform(testdf4['소득분위'].values)
testdf4['소득분류']=encoder.fit_transform(testdf4['소득분류'].values)
testdf4=testdf4.astype({'연간소득':'int','가족규모':'int'})
testdf4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   성별      10000 non-null  int64
 1   차       10000 non-null  int64
 2   집       10000 non-null  int64
 3   자녀 수    10000 non-null  int64
 4   연간소득    10000 non-null  int32
 5   소득분류    10000 non-null  int64
 6   교육수준    10000 non-null  int64
 7   결혼여부    10000 non-null  int64
 8   생활방식    10000 non-null  int64
 9   폰       10000 non-null  int64
 10  업무용 폰   10000 non-null  int64
 11  전화      10000 non-null  int64
 12  이메일     10000 non-null  int64
 13  직업      10000 non-null  int64
 14  가족규모    10000 non-null  int32
 15  소득분위    10000 non-null  int64
dtypes: int32(2), int64(14)
memory usage: 1.1 MB


In [295]:
testdf4.shape

(10000, 16)

In [294]:
y_card_df = carddrop['신용등급']
x_card_df = carddrop.drop(columns=['성별','생활방식','차'])

X_train,X_test,y_train,y_test=train_test_split(x_card_df,y_card_df,test_size=0.2,random_state=11)

ValueError: Found input variables with inconsistent numbers of samples: [10000, 26457]

In [282]:
#3개의 알고리즘을 위한 사이킷런의 클래스 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression()

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))

DecisionTreeClassifier 정확도: 0.6429
RandomForestClassifier 정확도: 0.6699
LogisticRegression 정확도: 0.6345
